<a href="https://colab.research.google.com/github/mahin567/yolov8-using-our-dataset/blob/main/criminal_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# --------------------------------------------------------------
# 1. Install packages (run once)
# --------------------------------------------------------------
!pip install -U pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install ultralytics==8.3.20 opencv-python-headless matplotlib pandas scikit-learn tqdm

Looking in indexes: https://download.pytorch.org/whl/cu118


In [8]:
# --------------------------------------------------------------
# 2. Extract the uploaded zip (test.zip)
# --------------------------------------------------------------
import os, zipfile, shutil

ZIP_NAME   = 'test.zip'                     # <-- your zip file name
EXTRACT_TO = '/content/test'                # <-- final dataset root

if not os.path.exists(f'/content/{ZIP_NAME}'):
    raise FileNotFoundError(f"{ZIP_NAME} not found in /content/. Upload it first!")

os.makedirs(EXTRACT_TO, exist_ok=True)

with zipfile.ZipFile(f'/content/{ZIP_NAME}', 'r') as z:
    z.extractall(EXTRACT_TO)

print(f"Extracted {ZIP_NAME} → {EXTRACT_TO}")

Extracted test.zip → /content/test


In [9]:
# --------------------------------------------------------------
# 3. Re-organise folders to the YOLO standard layout
# --------------------------------------------------------------
import shutil

# Source folders inside the zip
src_train_img = os.path.join(EXTRACT_TO, 'train', 'images')
src_train_lbl = os.path.join(EXTRACT_TO, 'train', 'labels')
src_val_img   = os.path.join(EXTRACT_TO, 'valid', 'images')
src_val_lbl   = os.path.join(EXTRACT_TO, 'valid', 'labels')
src_test_img  = os.path.join(EXTRACT_TO, 'test', 'images')
src_test_lbl  = os.path.join(EXTRACT_TO, 'test', 'labels')

# Destination folders (YOLO expects images/… and labels/…)
dst_img = os.path.join(EXTRACT_TO, 'images')
dst_lbl = os.path.join(EXTRACT_TO, 'labels')

os.makedirs(os.path.join(dst_img, 'train'), exist_ok=True)
os.makedirs(os.path.join(dst_img, 'val'),   exist_ok=True)
os.makedirs(os.path.join(dst_img, 'test'),  exist_ok=True)
os.makedirs(os.path.join(dst_lbl, 'train'), exist_ok=True)
os.makedirs(os.path.join(dst_lbl, 'val'),   exist_ok=True)
os.makedirs(os.path.join(dst_lbl, 'test'),  exist_ok=True)

# Helper to copy files
def copy_folder(src, dst):
    for f in os.listdir(src):
        shutil.copy(os.path.join(src, f), os.path.join(dst, f))

# Copy everything
copy_folder(src_train_img, os.path.join(dst_img, 'train'))
copy_folder(src_train_lbl, os.path.join(dst_lbl, 'train'))
copy_folder(src_val_img,   os.path.join(dst_img, 'val'))
copy_folder(src_val_lbl,   os.path.join(dst_lbl, 'val'))
copy_folder(src_test_img,  os.path.join(dst_img, 'test'))
copy_folder(src_test_lbl,  os.path.join(dst_lbl, 'test'))

print("Folder layout fixed:")
!find {EXTRACT_TO} -type d | head -n 20

Folder layout fixed:
/content/test
/content/test/train
/content/test/train/labels
/content/test/train/images
/content/test/labels
/content/test/labels/train
/content/test/labels/val
/content/test/labels/test
/content/test/valid
/content/test/valid/labels
/content/test/valid/images
/content/test/images
/content/test/images/train
/content/test/images/val
/content/test/images/test
/content/test/test
/content/test/test/labels
/content/test/test/images


In [10]:
# --------------------------------------------------------------
# 4. Create a correct data.yaml (absolute paths)
# --------------------------------------------------------------
yaml_path = os.path.join(EXTRACT_TO, 'data.yaml')

yaml_content = f"""path: {EXTRACT_TO}
train: images/train
val:   images/val
test:  images/test   # optional

nc: 2
names: ['with dangerous weapon', 'normal person']
"""

with open(yaml_path, 'w') as f:
    f.write(yaml_content.lstrip())

print(f"data.yaml created at: {yaml_path}")
!cat {yaml_path}

data.yaml created at: /content/test/data.yaml
path: /content/test
train: images/train
val:   images/val
test:  images/test   # optional

nc: 2
names: ['with dangerous weapon', 'normal person']


In [13]:
# --------------------------------------------------------------
# 5. Train YOLOv8n (FIXED: no invalid project name)
# --------------------------------------------------------------
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data=yaml_path,          # full path
    epochs=50,
    imgsz=420,
    batch=8,
    name='weapon_detection_v1',   # <-- this becomes the folder name
    exist_ok=True
    # REMOVED: project='/content/runs'  ← this caused the error
)

print("Training finished!")
print("Best model → runs/detect/weapon_detection_v1/weights/best.pt")

New https://pypi.org/project/ultralytics/8.3.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.20 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/test/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=420, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=weapon_detection_v1, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning /content/test/labels/train.cache... 15861 images, 53 backgrounds, 0 corrupt: 100%|██████████| 15861/15861 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 37, len(boxes) = 25887. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.12/dist-packages/ultralytics/data/augment.py:1850: UserWarning: Argument(s) 'quality_lower' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=75, p=0.0),
val: Scanning /content/test/labels/val.cache... 1897 images, 10 backgrounds, 0 corrupt: 100%|██████████| 1897/1897 [00:00<?, ?it/s]

...

                   all       1897       3285      0.877      0.852      0.908      0.713

50 epochs completed in 3.575 hours.
Optimizer stripped from runs/detect/weapon_detection_v1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/weapon_detection_v1/weights/best.pt, 6.2MB

Validating runs/detect/weapon_detection_v1/weights/best.pt...
Ultralytics 8.3.20 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 119/119 [00:17<00:00,  6.90it/s]


                   all       1897       3285      0.877      0.852      0.908      0.713
 with dangerous weapon       1885       2186      0.868      0.801      0.874      0.657
         normal person        819       1099      0.887      0.902      0.941      0.768
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/weapon_detection_v1


lr/pg0,▃▆███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg1,▃▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg2,▃▆███▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▃▁▃▃▄▃▅▆▆▇▇▇▇▇▇▇▇█▇▇███████████████████
metrics/mAP50-95(B),▂▃▁▃▃▄▃▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████
metrics/precision(B),▁▁▃▂▅▆▆▆▇▇█▇▆▆▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇██
metrics/recall(B),▁▃▂▃▃▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


Training finished!
Best model → runs/detect/weapon_detection_v1/weights/best.pt
